### Load Images and run Napari

In [ ]:
import napari
import numpy as np
from skimage.io import imread_collection
import glob

# Usa glob per ottenere correttamente i percorsi di tutte le immagini RGB
rgb_image_paths = sorted(glob.glob("../data/RGBintegrals/layer_*.png"))

# Carica le immagini RGB come stack 3D
rgb_images = imread_collection(rgb_image_paths).concatenate()

# Crea il viewer Napari con le immagini RGB
viewer = napari.Viewer()
viewer.add_image(rgb_images, name="RGB Integrals", rgb=True)

# Crea layer vuoto per annotazioni manuali
labels_layer = viewer.add_labels(np.zeros(rgb_images.shape[:-1], dtype=int), name="Manual Labels")

napari.run()


: 

In [3]:
import os
import glob
import torch
import numpy as np
import tifffile
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from skimage.io import imread
import napari                         # <-- NEW: per aggiornare/creare il layer di predizione

IMAGES_DIR = "../data/data1/min_results"
MODEL_DIR  = "models_iterative"
os.makedirs(MODEL_DIR, exist_ok=True)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")


# ---------------------------------------------------------------------
#  MODELLINO 2-CLASSI
# ---------------------------------------------------------------------
class Simple2ClassNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 2, 3, padding=1)
        self.relu  = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x


# ---------------------------------------------------------------------
#  LOSS: usa solo i pixel annotati (lbl == 2 o 3)
# ---------------------------------------------------------------------
def partial_cross_entropy(logits, lbls, min_w=1.0, max_w=200.0):
    unlabeled = (lbls == 0)
    # print(f"  Loss - Valori unici etichette input (lbls): {torch.unique(lbls)}") 
    target    = torch.where(lbls == 2, 1, 0)          # 2→1 (fuoco), 3→0 (sfocato)
    # print(f"  Loss - Valori unici target (0/1): {torch.unique(target)}")

    # raddrizza tensori
    logits = logits.permute(0, 2, 3, 1).reshape(-1, 2)
    target = target.reshape(-1)
    mask   = (~unlabeled.reshape(-1))

    # print(f"  Loss - Numero pixel etichettati (mask.sum()): {mask.sum().item()}")

    if not mask.any():
        return torch.tensor(0.0, device=logits.device, requires_grad=True)

    # pixel etichettati
    logits_lab = logits[mask]
    target_lab = target[mask]
    # print(f"  Loss - Shape target_lab: {target_lab.shape}")

    # ---- P E S I  D Y N A M I C I -----------------------------------
    n_total = target_lab.numel()
    n_pos   = (target_lab == 1).sum()
    n_neg   = n_total - n_pos

    # print(f"  Loss - n_total: {n_total}, n_pos (in-focus): {n_pos.item()}, n_neg (out-of-focus): {n_neg.item()}") # <-- CONTROLLA QUI!

    # evita divisione per zero
    if n_pos == 0 or n_neg == 0:
        weight = torch.tensor([1.0, 1.0], device=logits.device)
    else:
        # peso inversamente proporzionale alla frequenza
        w_neg = torch.clamp(n_total / (2.0 * n_neg.float()), min=min_w, max=max_w)
        w_pos = torch.clamp(n_total / (2.0 * n_pos.float()), min=min_w, max=max_w)
        weight = torch.tensor([w_neg, w_pos], device=logits.device)
        # print(f"  Loss - Pesi calcolati: {weight}")

    loss = F.cross_entropy(logits_lab, target_lab, weight=weight, reduction='mean')
    # print(f"  Loss - Valore loss calcolato: {loss.item()}") # <-- CONTROLLA QUI!
    return loss


# ---------------------------------------------------------------------
#  DATASET “one-shot” (un’unica immagine + mask)
# ---------------------------------------------------------------------
class NapariDataset(Dataset):
    def __init__(self, image, mask):
        self.image = image.astype(np.float32) / (255.0 if image.max() > 1 else 1.0)
        self.mask  = mask.astype(np.float32)

    def __len__(self):  return 1
    def __getitem__(self, idx):
        img = np.expand_dims(self.image, axis=0)       # (1, H, W)
        return torch.from_numpy(img), torch.from_numpy(self.mask)


# ---------------------------------------------------------------------
#  TRAIN O FINE-TUNE DEL MODELLO PER UN LAYER
# ---------------------------------------------------------------------
def train_model(layer_idx, img, mask, model_path, epochs=100, lr=1e-5):
    model = Simple2ClassNet().to(device)

    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"[Layer {layer_idx}] modello caricato, proseguo fine-tuning.")
    else:
        print(f"[Layer {layer_idx}] nuovo modello creato.")

    loader = DataLoader(NapariDataset(img, mask), batch_size=1, shuffle=False)
    optim  = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for ep in range(epochs):
        for imgs, lbls in loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            optim.zero_grad()
            loss = partial_cross_entropy(model(imgs), lbls)
            loss.backward()
            optim.step()
        # print(f"[Layer {layer_idx}] epoch {ep+1}/{epochs} – loss {loss.item():.5f}")

    torch.save(model.state_dict(), model_path)
    return model


# ---------------------------------------------------------------------
#  INFERENZA SINGOLO LAYER
# ---------------------------------------------------------------------
@torch.no_grad()
def infer_mask(model, img):
    t = torch.from_numpy(img).unsqueeze(0).unsqueeze(0).float().to(device)
    pred = torch.softmax(model(t), dim=1).argmax(1).squeeze(0).cpu().numpy()
    # print(f"  Inferenza - Valori unici nella predizione: {np.unique(pred)}") # <-- CONTROLLA QUI!
    return pred   # 0 = out-of-focus, 1 = in-focus


# ---------------------------------------------------------------------
#  ---- MAIN LOOP ----
# ---------------------------------------------------------------------
labels_layer        = napari.current_viewer().layers['Manual Labels']
labels_data         = labels_layer.data                  # (Z, H, W) con 0/2/3
variance_paths      = sorted(glob.glob(f"{IMAGES_DIR}/variance_image_*_min_grayscale.tiff"))
num_layers, H, W    = labels_data.shape
final_prediction    = np.zeros_like(labels_data)         # (Z, H, W) 0/1

for z in range(num_layers):
    mask   = labels_data[z]
    # print(f"[Layer {z}] Valori unici nella maschera manuale: {np.unique(mask)}")
    img    = imread(variance_paths[max(z, 0)]).astype(np.float32) / 255.0
    # print(f"[Layer {z}] Immagine caricata - min: {img.min()}, max: {img.max()}, mean: {img.mean()}") 
    mpath  = os.path.join(MODEL_DIR, f"layer_{z}.pt")

    # train se ci sono annotazioni, altrimenti prendi modello “più vicino”
    if np.any(mask > 0):
        model = train_model(z, img, mask, mpath)
    else:
        available = [i for i in range(num_layers) if os.path.exists(os.path.join(MODEL_DIR, f"layer_{i}.pt"))]
        if not available:
            # print(f"[Layer {z}] nessun modello disponibile - skippato.")
            continue
        nearest = min(available, key=lambda i: abs(z - i))
        mpath   = os.path.join(MODEL_DIR, f"layer_{nearest}.pt")
        model   = Simple2ClassNet().to(device)
        model.load_state_dict(torch.load(mpath, map_location=device))
        # print(f"[Layer {z}] uso modello del layer {nearest}.")

    final_prediction[z] = infer_mask(model, img)         # 0/1


# ---------------------------------------------------------------------
#  -----  AGGIORNA / CREA LAYER “Model Pred” IN NAPARI  -----
# ---------------------------------------------------------------------
viewer  = napari.current_viewer()
PRED_L  = 'Model Pred'

if PRED_L in viewer.layers:
    viewer.layers[PRED_L].data = final_prediction          # aggiorna dati
else:
    # 1) crea il layer (senza 'color='!)
    pred_layer = viewer.add_labels(
        final_prediction,
        name    = PRED_L,
        opacity = 0.40,        # trasparenza
    )

    # 2) imposta la tavolozza colori a posteriori
    #    (0 = sfocato → magenta, 1 = a fuoco → lime)
    pred_layer.color = {0: 'magenta', 1: 'lime'}

    # 3) blocca l’editing per evitare tocchi accidentali
    pred_layer.editable = False

print("✓ Predizione aggiornata sul layer 'Model Pred'. "
      "Accendi/spegni la trasparenza per controllare gli errori e "
      "correggi soltanto sul layer 'Manual Labels'.")



[Layer 0] modello caricato, proseguo fine-tuning.


KeyboardInterrupt: 

## Inference different from zero

In [ ]:
import napari
import numpy as np

try:
    # Ottieni il viewer Napari corrente
    viewer = napari.current_viewer()

    if viewer is None:
        print("Errore: Nessun viewer Napari attivo.")
    else:
        layer_name = 'Model Pred'
        if layer_name in viewer.layers:
            # Accedi al layer delle predizioni
            prediction_layer = viewer.layers[layer_name]

            # Ottieni i dati (l'array NumPy con shape (Z, H, W))
            prediction_data = prediction_layer.data

            # --- Modifica per contare i layer con valori > 0 ---
            num_total_layers = prediction_data.shape[0]
            layers_with_non_zeros_count = 0
            indices_with_non_zeros = [] # Lista per tenere traccia degli indici (opzionale)

            # Itera su ogni singolo layer (slice 2D)
            for i in range(num_total_layers):
                # Prendi i dati del layer corrente
                current_layer_data = prediction_data[i]
                # Controlla se ESISTE ALMENO UN valore > 0 in questo layer
                if np.any(current_layer_data > 0):
                    layers_with_non_zeros_count += 1
                    indices_with_non_zeros.append(i) # Aggiunge l'indice del layer alla lista

            # --- Fine Modifica ---

            # Stampa il riepilogo
            print(f"Controllo '{layer_name}':")
            print(f"- Numero totale di layer: {num_total_layers}")

            if layers_with_non_zeros_count > 0:
                print(f"- Numero di layer contenenti valori diversi da 0: {layers_with_non_zeros_count}")
                # Puoi anche stampare gli indici se ti interessa sapere *quali* layer sono
                print(f"- Indici dei layer con valori > 0: {indices_with_non_zeros}")

                # Conferma i valori unici globali trovati
                unique_values = np.unique(prediction_data)
                print(f"- Valori unici trovati nell'intero stack: {unique_values}")
            else:
                # Questo caso non dovrebbe verificarsi dato il tuo output precedente, ma lo teniamo per completezza
                print("- Tutti i layer contengono solo il valore 0.")

        else:
            print(f"Errore: Il layer '{layer_name}' non esiste nel viewer.")
            print("Assicurati di aver eseguito lo script di training/predizione.")

except Exception as e:
    print(f"Si è verificato un errore durante l'accesso a Napari o al layer: {e}")

## Train a Unique Model

In [5]:
import os
import glob
import torch
import numpy as np
import tifffile
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from skimage.io import imread
import napari
import time # Per misurare il tempo

# --- Assicurati che queste siano definite prima ---
# class Simple2ClassNet(nn.Module): ...
# def partial_cross_entropy(logits, lbls, min_w=1.0, max_w=50.0): ... # Nota: max_w aumentato!
# @torch.no_grad() def infer_mask(model, img): ...
# -------------------------------------------------

# --- Configurazioni ---
IMAGES_DIR = "../data/data1/min_results"
MODEL_DIR  = "models_iterative"
os.makedirs(MODEL_DIR, exist_ok=True)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Nuovo nome per il layer di output
UNIQUE_PRED_LAYER_NAME = 'Model Pred Unique'

# --- Dataset Modificato per gestire Multipli Layer ---
class NapariMultiLayerDataset(Dataset):
    def __init__(self, images_list, masks_list):
        # Normalizza tutte le immagini in input
        self.images = []
        for img in images_list:
             # Assicurati che sia float32 e normalizzato
             img_float = img.astype(np.float32)
             # Normalizza dividendo per 255 se i valori sono alti, altrimenti per 1
             norm_factor = 255.0 if img.max() > 1.5 else 1.0 # Leggermente più robusto di "> 1"
             self.images.append(img_float / norm_factor)

        # Le maschere rimangono come sono (int o float, la loss le gestirà)
        self.masks = [m.astype(np.int64) for m in masks_list] # Usa int64 per compatibilità loss PyTorch

    def __len__(self):
        # La lunghezza è il numero di coppie immagine/maschera fornite
        return len(self.images)

    def __getitem__(self, idx):
        # Prende l'immagine e la maschera all'indice specificato
        img = self.images[idx]
        mask = self.masks[idx]

        # Aggiunge la dimensione del canale all'immagine (richiesta da Conv2d)
        img_tensor = torch.from_numpy(np.expand_dims(img, axis=0)) # Shape: (1, H, W)
        mask_tensor = torch.from_numpy(mask) # Shape: (H, W)

        return img_tensor, mask_tensor

# --- 1. Raccogli TUTTI i dati annotati ---
print("--- Step 1: Collecting Annotated Data ---")
try:
    viewer = napari.current_viewer()
    if viewer is None:
        raise RuntimeError("Nessun viewer Napari attivo.")

    labels_layer = viewer.layers['Manual Labels']
    labels_data = labels_layer.data # Shape (Z, H, W)
    variance_paths = sorted(glob.glob(f"{IMAGES_DIR}/variance_image_*_min_grayscale.tiff"))
    num_layers, H, W = labels_data.shape

    if len(variance_paths) != num_layers:
         print(f"Warning: Numero di immagini ({len(variance_paths)}) non corrisponde al numero di layer nelle etichette ({num_layers}). Potrebbero esserci errori.")

    all_images_to_train = []
    all_masks_to_train = []
    annotated_layer_indices = []

    print("Scanning layers for annotations...")
    for z in range(num_layers):
        mask = labels_data[z]
        # Controlla se ci sono etichette DIVERSE da 0
        if np.any(mask != 0):
            # Assicurati che ci siano etichette 2 o 3 (o entrambe)
            unique_labels_in_mask = np.unique(mask)
            if 2 in unique_labels_in_mask or 3 in unique_labels_in_mask:
                try:
                    # Carica l'immagine corrispondente
                    img = imread(variance_paths[z])
                    all_images_to_train.append(img) # Normalizzazione fatta nel Dataset
                    all_masks_to_train.append(mask)
                    annotated_layer_indices.append(z)
                    # print(f"  Layer {z}: Found annotations {unique_labels_in_mask}. Added to training set.")
                except IndexError:
                    print(f"Warning: Impossibile trovare variance_paths[{z}]. Salto layer {z} annotato.")
                except FileNotFoundError:
                     print(f"Warning: Immagine varianza non trovata per layer {z} annotato: {variance_paths[z]}. Salto.")
            # else:
            #    print(f"  Layer {z}: Contiene etichette diverse da 0, ma non 2 o 3 ({unique_labels_in_mask}). Escluso.")
        # else:
        #    print(f"  Layer {z}: Nessuna annotazione manuale trovata.")


    if not all_images_to_train:
        print("\nErrore: Nessun layer con annotazioni valide (2 o 3) trovato. Impossibile addestrare.")
        # Qui potresti voler uscire o gestire l'errore diversamente
    else:
        print(f"\nRaccolti {len(all_images_to_train)} layer annotati per l'addestramento: {annotated_layer_indices}")

except Exception as e:
    print(f"Errore durante la raccolta dati: {e}")
    # Esci o gestisci l'errore se non puoi continuare
    all_images_to_train = [] # Assicura che non si proceda se c'è stato un errore

# --- 2. Addestra UN SINGOLO Modello ---
print("\n--- Step 2: Training Single Model ---")
model = None # Inizializza a None
unique_model_path = os.path.join(MODEL_DIR, "unique_focus_model.pt") # Nome file specifico

if all_images_to_train: # Procedi solo se hai dati
    epochs = 60 # Potrebbero servire più epoche per un dataset aggregato
    lr = 1e-3
    batch_size = 4 # Esempio; aggiusta in base alla memoria GPU/CPU
    # Usa un valore alto per max_w come discusso!
    loss_max_w = 50.0

    # Istanzia il modello
    model = Simple2ClassNet().to(device)

    # Carica il modello se esiste già per fine-tuning
    if os.path.exists(unique_model_path):
        try:
            model.load_state_dict(torch.load(unique_model_path, map_location=device))
            print(f"Caricato modello esistente da: {unique_model_path}. Si procede al fine-tuning.")
        except Exception as e:
            print(f"Errore nel caricamento del modello da {unique_model_path}. Si addestra da zero. Dettagli: {e}")
    else:
        print(f"Nessun modello pre-esistente trovato. Creazione nuovo modello unico.")

    # Crea Dataset e DataLoader
    dataset = NapariMultiLayerDataset(all_images_to_train, all_masks_to_train)
    # Shuffle=True è importante per l'addestramento
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0) # num_workers=0 per MPS

    # Ottimizzatore
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    print(f"Inizio addestramento su {len(dataset)} campioni (layer annotati) per {epochs} epoche...")
    start_time_train = time.time()

    for ep in range(epochs):
        model.train() # Imposta modalità training
        epoch_loss_sum = 0.0
        batches_processed = 0
        for i, (imgs_batch, lbls_batch) in enumerate(loader):
            # imgs_batch: (B, 1, H, W), lbls_batch: (B, H, W)
            imgs_batch, lbls_batch = imgs_batch.to(device), lbls_batch.to(device)

            optim.zero_grad()
            logits_batch = model(imgs_batch) # Output: (B, 2, H, W)

            # Calcola la loss sul batch, usando i pesi aumentati
            loss = partial_cross_entropy(logits_batch, lbls_batch, max_w=loss_max_w)

            # Propaga l'errore solo se la loss è valida (es. se c'erano etichette nel batch)
            if loss.requires_grad:
                loss.backward()
                optim.step()
                epoch_loss_sum += loss.item()
                batches_processed += 1
            # else:
                # print(f"  Batch {i}: Loss non richiede gradiente (probabilmente nessuna etichetta nel batch?).")


        avg_epoch_loss = epoch_loss_sum / batches_processed if batches_processed > 0 else 0
        print(f"Epoch {ep+1}/{epochs} - Avg Loss: {avg_epoch_loss:.6f}")

    end_time_train = time.time()
    print(f"Addestramento completato in {end_time_train - start_time_train:.2f} secondi.")

    # Salva il modello unico addestrato
    try:
        torch.save(model.state_dict(), unique_model_path)
        print(f"Modello unico salvato in: {unique_model_path}")
    except Exception as e:
        print(f"Errore durante il salvataggio del modello: {e}")

else:
    print("Addestramento saltato perché non sono stati raccolti dati annotati validi.")
    # Prova a caricare il modello se esiste, altrimenti l'inferenza fallirà
    if os.path.exists(unique_model_path):
         model = Simple2ClassNet().to(device)
         model.load_state_dict(torch.load(unique_model_path, map_location=device))
         print(f"Modello caricato da {unique_model_path} per inferenza (addestramento saltato).")


# --- 3. Esegui Inferenza su TUTTI i Layer con il Modello Unico ---
print("\n--- Step 3: Performing Inference ---")
# Assicurati che esista un modello (o addestrato ora o caricato)
if model is None:
     print("Errore: Nessun modello disponibile per l'inferenza.")
else:
    model.eval() # Imposta modalità valutazione (importante!)
    final_prediction = np.zeros_like(labels_data, dtype=np.int8) # Maschera 0/1

    print(f"Applicazione del modello unico a tutti i {num_layers} layer...")
    start_time_infer = time.time()

    for z in range(num_layers):
        try:
            # Carica e normalizza l'immagine di varianza per il layer corrente
            img = imread(variance_paths[z]).astype(np.float32)
            img_norm = img / (255.0 if img.max() > 1.5 else 1.0)

            # Esegui inferenza con il modello unico
            prediction_mask = infer_mask(model, img_norm) # infer_mask ritorna array numpy 0/1
            final_prediction[z] = prediction_mask.astype(np.int8)

            # Stampa progresso ogni 10 layer (opzionale)
            # if (z + 1) % 10 == 0 or z == num_layers - 1:
            #     print(f"  Inferenza completata per layer {z+1}/{num_layers}")

        except (FileNotFoundError, IndexError):
            print(f"Warning: Immagine varianza per layer {z} non trovata o indice errato. Inferenza saltata.")
            final_prediction[z] = 0 # Lascia il layer a 0 se l'immagine manca
        except Exception as e:
            print(f"Errore durante l'inferenza per layer {z}: {e}. Inferenza saltata.")
            final_prediction[z] = 0

    end_time_infer = time.time()
    print(f"Inferenza completata in {end_time_infer - start_time_infer:.2f} secondi.")

    # --- 4. Aggiorna/Crea Layer in Napari ---
    print(f"\n--- Step 4: Updating Napari Layer '{UNIQUE_PRED_LAYER_NAME}' ---")
    if viewer: # Controlla se il viewer esiste ancora
        if UNIQUE_PRED_LAYER_NAME in viewer.layers:
            print(f"Aggiornamento layer esistente '{UNIQUE_PRED_LAYER_NAME}'...")
            try:
                viewer.layers[UNIQUE_PRED_LAYER_NAME].data = final_prediction
                # Riapplica colore ed editabilità per sicurezza
                viewer.layers[UNIQUE_PRED_LAYER_NAME].color = {0: 'magenta', 1: 'lime'}
                viewer.layers[UNIQUE_PRED_LAYER_NAME].editable = False
                print("Layer aggiornato.")
            except Exception as e:
                print(f"Errore durante l'aggiornamento del layer: {e}")
        else:
            print(f"Creazione nuovo layer '{UNIQUE_PRED_LAYER_NAME}'...")
            try:
                pred_layer = viewer.add_labels(
                    final_prediction,
                    name=UNIQUE_PRED_LAYER_NAME,
                    opacity=0.40,
                )
                # Imposta colore e editabilità
                pred_layer.color = {0: 'magenta', 1: 'lime'}
                pred_layer.editable = False
                print("Nuovo layer creato.")
            except Exception as e:
                 print(f"Errore durante la creazione del layer: {e}")

        print(f"\n✓ Predizione 'unica' aggiornata sul layer '{UNIQUE_PRED_LAYER_NAME}'.")
    else:
        print("Viewer Napari non trovato per aggiornare il layer.")

Using device: mps
--- Step 1: Collecting Annotated Data ---
Scanning layers for annotations...

Raccolti 7 layer annotati per l'addestramento: [0, 10, 15, 17, 33, 36, 45]

--- Step 2: Training Single Model ---
Caricato modello esistente da: models_iterative/unique_focus_model.pt. Si procede al fine-tuning.
Inizio addestramento su 7 campioni (layer annotati) per 60 epoche...
Epoch 1/60 - Avg Loss: 0.543709
Epoch 2/60 - Avg Loss: 0.445135
Epoch 3/60 - Avg Loss: 0.537893
Epoch 4/60 - Avg Loss: 0.536940
Epoch 5/60 - Avg Loss: 0.486156
Epoch 6/60 - Avg Loss: 0.467444
Epoch 7/60 - Avg Loss: 0.536655
Epoch 8/60 - Avg Loss: 0.494550
Epoch 9/60 - Avg Loss: 0.528136
Epoch 10/60 - Avg Loss: 0.505929
Epoch 11/60 - Avg Loss: 0.537301
Epoch 12/60 - Avg Loss: 0.534794
Epoch 13/60 - Avg Loss: 0.505544
Epoch 14/60 - Avg Loss: 0.473211
Epoch 15/60 - Avg Loss: 0.526929
Epoch 16/60 - Avg Loss: 0.504592
Epoch 17/60 - Avg Loss: 0.518113
Epoch 18/60 - Avg Loss: 0.546298
Epoch 19/60 - Avg Loss: 0.505221
Epoch

## Video Maker

In [ ]:
import napari
import numpy as np
import os
import time # Import time just in case needed, though not strictly required by logic

# Prova ad importare cv2 e tqdm, gestendo l'assenza
try:
    import cv2 # Per la scrittura video
    print("Libreria OpenCV (cv2) trovata.")
except ImportError:
    print("--------------------------------------------------------------------")
    print("ERRORE: La libreria 'opencv-python' (cv2) non è installata.")
    print("Questa libreria è necessaria per creare il video.")
    print("Per favore, installala dal tuo terminale/console eseguendo:")
    print("  pip install opencv-python")
    print("--------------------------------------------------------------------")
    # Imposta cv2 a None per interrompere l'esecuzione più avanti se manca
    cv2 = None

try:
    from tqdm.auto import tqdm # Per la barra di progresso (opzionale)
    print("Libreria tqdm trovata (verrà mostrato il progresso).")
except ImportError:
    print("Info: Libreria 'tqdm' non trovata. Il progresso non verrà mostrato.")
    # Definisci una funzione tqdm "finta" se non è installata
    # così il codice non dà errore ma semplicemente itera senza barra
    def tqdm(iterator, *args, **kwargs):
        print("Inizio elaborazione frame...")
        return iterator
    tqdm = tqdm # Assegna la funzione finta

# --- Parametri di Configurazione ---

# <<<======================================================================>>>
# <<<    MODIFICA QUI il nome del layer da cui vuoi creare il video       >>>
# <<<======================================================================>>>
layer_name_to_process = "Model Pred Unique"  # Oppure cambia in "Model Pred"

output_directory = "output_videos" # Cartella dove salvare il video
# Crea un nome file basato sul nome del layer, sostituendo spazi
safe_layer_name = layer_name_to_process.replace(' ', '_')
output_filename = f"{safe_layer_name}_BW_video.mp4"
output_path = os.path.join(output_directory, output_filename)

fps = 10.0 # Frame al secondo del video finale (puoi aggiustare)

# --- Inizio Script ---

print(f"\nTentativo di creare un video in Bianco/Nero per il layer: '{layer_name_to_process}'")
print(f"Il video verrà salvato come: '{output_path}' a {int(fps)} FPS.")

# Procedi solo se cv2 è stato importato correttamente
if cv2 is not None:
    video_writer = None # Inizializza a None per il blocco finally
    try:
        # 1. Ottieni i Dati dal Layer Napari
        print("Accesso a Napari...")
        viewer = napari.current_viewer()
        if viewer is None:
            raise RuntimeError("Nessun viewer Napari attivo trovato.")

        if layer_name_to_process not in viewer.layers:
            raise KeyError(f"Il layer specificato '{layer_name_to_process}' non è presente nel viewer Napari.")

        print(f"Accesso al layer '{layer_name_to_process}'...")
        prediction_layer = viewer.layers[layer_name_to_process]
        prediction_data = prediction_layer.data # Dovrebbe essere un array (Z, H, W) con 0 e 1

        # Controlli sui dati
        if not isinstance(prediction_data, np.ndarray):
             raise TypeError("I dati del layer non sono un array NumPy.")
        if prediction_data.ndim != 3:
             raise ValueError(f"I dati del layer devono avere 3 dimensioni (Z, H, W). Trovate: {prediction_data.ndim}")

        num_frames, H, W = prediction_data.shape
        print(f"Dati caricati: {num_frames} frames (layers), Dimensioni: {H}x{W}")

        unique_vals = np.unique(prediction_data)
        print(f"Valori unici trovati nei dati del layer: {unique_vals}")
        # Avvisa se ci sono valori inaspettati, ma prova comunque
        if not np.all(np.isin(unique_vals, [0, 1])):
             print("ATTENZIONE: I dati contengono valori diversi da 0 e 1. Verranno mappati come segue: 0->Nero, >0->Bianco.")

        # 2. Prepara il Video Writer di OpenCV
        # Crea la cartella di output se non esiste
        os.makedirs(output_directory, exist_ok=True)

        # Definisci il codec (FOURCC) per il formato video.
        # 'mp4v' è un buon default per .mp4, compatibile con molti sistemi.
        # Altri comuni: 'XVID' per .avi
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Crea l'oggetto VideoWriter. Nota l'ordine (W, H) per frameSize.
        # isColor=False perché il nostro output è binario (1 canale).
        print(f"Inizializzazione video writer per '{output_path}'...")
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, (W, H), isColor=False)

        # Verifica se il VideoWriter è stato aperto correttamente
        if not video_writer.isOpened():
             # Fornisce un messaggio di errore più dettagliato se possibile
             codec_info = "Codec 'mp4v' potrebbe non essere supportato o mancare librerie." if 'mp4v' in 'mp4v' else "Problema con il codec specificato."
             raise IOError(f"Errore critico: Impossibile aprire il file video '{output_path}' per la scrittura. {codec_info} Verifica permessi.")

        # 3. Elabora i Frame e Scrivi il Video
        print(f"\nInizio elaborazione e scrittura dei {num_frames} frames...")
        # Itera su ogni slice (layer) lungo l'asse Z (asse 0)
        # Usa tqdm per mostrare una barra di progresso se disponibile
        for i in tqdm(range(num_frames), desc=f"Creazione video"):
            # Estrai la slice 2D corrente
            layer_slice = prediction_data[i] # Contiene 0 o 1

            # Converti la slice in un frame uint8 Bianco e Nero
            # Mappa: 0 -> 0 (nero), 1 -> 255 (bianco)
            # Qualsiasi valore > 0 verrà mappato a bianco con questo metodo.
            frame_bw = np.where(layer_slice > 0, 255, 0).astype(np.uint8)
            # Alternativa (se sei sicuro che ci siano solo 0 e 1):
            # frame_bw = (layer_slice * 255).astype(np.uint8)

            # Scrivi il frame nel file video
            video_writer.write(frame_bw)

        # 4. Rilascia le Risorse
        print("\nFinalizzazione video...")
        video_writer.release() # Chiude il file video correttamente
        print("Video creato con successo!")
        # Stampa il percorso assoluto per chiarezza
        print(f"File salvato in: {os.path.abspath(output_path)}")

    # Gestione degli errori specifici e generici
    except (KeyError, ValueError, TypeError, RuntimeError, IOError) as e:
         print(f"\n--- ERRORE ---")
         print(f"{type(e).__name__}: {e}")
         print("Creazione del video interrotta.")
    except Exception as e:
        print(f"\n--- ERRORE INASPETTATO ---")
        print(f"{type(e).__name__}: {e}")
        print("Creazione del video interrotta.")
    finally:
        # Assicurati SEMPRE di rilasciare il writer se è stato aperto,
        # anche se si è verificato un errore durante la scrittura dei frame.
        if video_writer is not None and video_writer.isOpened():
            print("Rilascio precauzionale del video writer...")
            video_writer.release()

else:
    # Messaggio se cv2 non era disponibile all'inizio
    print("\nOperazione annullata perché la libreria 'opencv-python' non è disponibile.")

- Create a box with 3 buttons: initialization - training/inference - make video
- No hard code, providing the name of the files (maybe also the saving path)
- getting better results
- change colors for daltonism
- add a 3D views (see document) --> also include this into UI
- save final results (inference)